In [29]:
import pandas as pd
from google.cloud import storage

# Trace Data Loading and Preprocessing

In [30]:
bucket_name = "trace_data_ic"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)


In [31]:

# blobs = bucket.list_blobs()

# for blob in blobs:
#     file_name = blob.name
#     print(file_name)
#     if file_name == "mergent_final_6k.pkl" or file_name == "trace_6k_bonds.gz" :
#         blob.download_to_filename(file_name) # download the file to the machine
    

In [32]:
# Loading the data into chunks of 10 Mil rows for processing
chunk_size = 10000000

# Initialize an empty DataFrame to store the final result
data_raw = pd.DataFrame()
i = 0 

# The file downloaded from WRDS on the basis of the CUSIP IDs from Mergent FISD Data
file = "trace_6k_bonds.gz"
# Load the CSV file into a DataFrame in chunks and process each chunk individually
for chunk in pd.read_csv(file, header=0, compression="gzip", chunksize=chunk_size):
        i+=1
        if i == 1:
            print(chunk.columns)
        print("Chunk:",i)
        # Concatenate the processed chunk with the existing data
        data_raw = pd.concat([data_raw, chunk], ignore_index=True)

# Reset the index of the concatenated DataFrame
data_raw.reset_index(drop=True, inplace=True)
data_raw.shape

Index(['cusip_id', 'bond_sym_id', 'company_symbol', 'trd_exctn_dt',
       'trd_exctn_tm', 'trd_rpt_dt', 'trd_rpt_tm', 'msg_seq_nb', 'trc_st',
       'entrd_vol_qt', 'rptd_pr', 'yld_pt', 'asof_cd', 'rpt_side_cd',
       'orig_msg_seq_nb', 'sub_prdct'],
      dtype='object')
Chunk: 1
Chunk: 2
Chunk: 3
Chunk: 4
Chunk: 5
Chunk: 6


(58022788, 16)

In [33]:
data_raw['cusip_id'].nunique()

6005

In [34]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58022788 entries, 0 to 58022787
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   cusip_id         object 
 1   bond_sym_id      object 
 2   company_symbol   object 
 3   trd_exctn_dt     object 
 4   trd_exctn_tm     object 
 5   trd_rpt_dt       object 
 6   trd_rpt_tm       object 
 7   msg_seq_nb       int64  
 8   trc_st           object 
 9   entrd_vol_qt     float64
 10  rptd_pr          float64
 11  yld_pt           float64
 12  asof_cd          object 
 13  rpt_side_cd      object 
 14  orig_msg_seq_nb  float64
 15  sub_prdct        object 
dtypes: float64(4), int64(1), object(11)
memory usage: 6.9+ GB


In [35]:
mergent_data = pd.read_pickle("mergent_final_6k.pkl")
mergent_data.shape

(6087, 64)

In [36]:
mergent_data.columns

Index(['issue_id', 'issuer_id', 'prospectus_issuer_name', 'issuer_cusip',
       'issue_cusip', 'issue_name', 'maturity', 'security_level',
       'enhancement', 'coupon_type', 'mtn', 'asset_backed', 'yankee',
       'canadian', 'oid', 'slob', 'issue_offered_global', 'settlement_type',
       'gross_spread', 'selling_concession', 'comp_neg_exch_deal',
       'rule_415_reg', 'sec_reg_type1', 'offering_amt', 'offering_date',
       'offering_price', 'offering_yield', 'delivery_date', 'unit_deal',
       'form_of_own', 'denomination', 'principal_amt', 'covenants', 'defeased',
       'defaulted', 'tender_exch_offer', 'redeemable', 'refund_protection',
       'overallotment_opt', 'announced_call', 'active_issue',
       'dep_eligibility', 'bond_type', 'subsequent_data', 'isin', 'fungible',
       'complete_cusip', 'action_type', 'effective_date', 'action_price',
       'action_amount', 'amount_outstanding', 'greater_of', 'lesser_of',
       'see_note', 'dated_date', 'first_interest_date', '

In [37]:
# Filtering the necessary Bond characteristics from the Mergent Data

bond_info = mergent_data[[
        'maturity', 'coupon_type', 'offering_amt', 'offering_date',
       'offering_price', 'offering_yield', 'delivery_date', 'principal_amt', 
        'bond_type','complete_cusip', 'coupon','years_to_maturity']]

In [38]:
bond_info.shape

(6087, 12)

In [39]:
# Merge the trace data and mergent data based on the 'cusip_id' column
merged_df = pd.merge(data_raw, bond_info,left_on='cusip_id', right_on='complete_cusip', how='inner')
merged_df.shape

(58022788, 28)

In [40]:
merged_df['cusip_id'].nunique()

6005

In [41]:
merged_df['bond_type'].value_counts()

CDEB    58022788
Name: bond_type, dtype: int64

In [42]:
merged_df['cusip_id'].nunique()

6005

# Applying Filters by Dick N.

In [43]:
# Start of the program

# Takes out all cancellations and corrections;
# These transactions should be deleted together with the
# original report;
print('Takes out all cancellations and corrections')
temp_deleteI_NEW = data_raw[data_raw['trc_st'].isin(['C', 'X'])]
temp_deleteI_NEW.info()

# Reversals. These have to be deleted as well together with
# the original report;
print('Reversals')
temp_deleteII_NEW = data_raw[data_raw['trc_st'].isin(['Y'])]
temp_deleteII_NEW.info()

# The rest of the data;
print('The rest of the data')
temp_raw = data_raw[~data_raw['trc_st'].isin(['C', 'X', 'Y'])]
temp_raw.info()

# Deletes the cancellations and corrections as identified by
# the reports in temp_deleteI_NEW;
# These transactions can be matched by message sequence number
# and date. We furthermore match on cusip, volume, price, date,
# time, buy-sell side, contra party;
# This is as suggested by the variable description;

cols_to_match = ['cusip_id', 'entrd_vol_qt', 'rptd_pr', 'trd_exctn_dt', 'trd_exctn_tm', 'rpt_side_cd', 'msg_seq_nb']
temp_raw2 = temp_raw.merge(temp_deleteI_NEW[cols_to_match], on=cols_to_match, how='left', indicator=True)
temp_raw2 = temp_raw2[temp_raw2['_merge'] == 'left_only'].drop('_merge', axis=1)
print('Deletes the cancellations and corrections')
temp_raw2.info()

# Deletes the reports that are matched by the reversals;

temp_raw3 = temp_raw2.merge(temp_deleteII_NEW[cols_to_match], on=cols_to_match, how='left', indicator=True)
temp_raw3 = temp_raw3[temp_raw3['_merge'] == 'left_only'].drop('_merge', axis=1)
print('Deletes the reports that are matched by the reversals')
temp_raw3.info()

# Ends the filtering of the post-change data
# clean used columns of data

temp_raw3_col = temp_raw3.drop(['asof_cd', 'trd_rpt_dt', 'trd_rpt_tm', 'msg_seq_nb', 'trc_st', 'orig_msg_seq_nb'], axis=1)


temp_raw3_col.shape

Takes out all cancellations and corrections
<class 'pandas.core.frame.DataFrame'>
Int64Index: 767093 entries, 102 to 58022703
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   cusip_id         767093 non-null  object 
 1   bond_sym_id      766893 non-null  object 
 2   company_symbol   766893 non-null  object 
 3   trd_exctn_dt     767093 non-null  object 
 4   trd_exctn_tm     767093 non-null  object 
 5   trd_rpt_dt       767093 non-null  object 
 6   trd_rpt_tm       767093 non-null  object 
 7   msg_seq_nb       767093 non-null  int64  
 8   trc_st           767093 non-null  object 
 9   entrd_vol_qt     767093 non-null  float64
 10  rptd_pr          767093 non-null  float64
 11  yld_pt           661138 non-null  float64
 12  asof_cd          185601 non-null  object 
 13  rpt_side_cd      767093 non-null  object 
 14  orig_msg_seq_nb  28240 non-null   float64
 15  sub_prdct        767093 non-null  obj

(56473882, 10)

In [44]:
temp_raw3_col['cusip_id'].nunique()

6004

In [45]:
temp_raw3_col.head()

,cusip_id,bond_sym_id,company_symbol,trd_exctn_dt,trd_exctn_tm,entrd_vol_qt,rptd_pr,yld_pt,rpt_side_cd,sub_prdct
0,00217GAA1,NaN,NaN,2022-02-09,15:29:00,1000000.0,100.0,NaN,S,CORP
1,00217GAA1,NaN,NaN,2022-02-09,15:29:00,2000000.0,100.0,NaN,S,CORP
2,00217GAA1,NaN,NaN,2022-02-09,15:29:00,1000000.0,100.0,NaN,S,CORP
3,00217GAA1,NaN,NaN,2022-02-09,15:29:00,400000.0,100.0,NaN,S,CORP
4,00217GAA1,NaN,NaN,2022-02-09,15:29:00,1000000.0,100.0,NaN,S,CORP


In [46]:
del temp_raw3

In [47]:
del temp_raw2

In [48]:
del temp_deleteII_NEW
del temp_deleteI_NEW

In [49]:
del merged_df

In [50]:
del data_raw

In [51]:
del temp_raw

In [52]:
final_df = temp_raw3_col.copy()
del temp_raw3_col

In [53]:
final_df.columns

Index(['cusip_id', 'bond_sym_id', 'company_symbol', 'trd_exctn_dt',
       'trd_exctn_tm', 'entrd_vol_qt', 'rptd_pr', 'yld_pt', 'rpt_side_cd',
       'sub_prdct'],
      dtype='object')

In [54]:
for col in final_df:
    print(col)
    print(final_df[col].isna().value_counts())

cusip_id
False    56473882
Name: cusip_id, dtype: int64
bond_sym_id
False    56470031
True         3851
Name: bond_sym_id, dtype: int64
company_symbol
False    56470031
True         3851
Name: company_symbol, dtype: int64
trd_exctn_dt
False    56473882
Name: trd_exctn_dt, dtype: int64
trd_exctn_tm
False    56473882
Name: trd_exctn_tm, dtype: int64
entrd_vol_qt
False    56473882
Name: entrd_vol_qt, dtype: int64
rptd_pr
False    56473882
Name: rptd_pr, dtype: int64
yld_pt
False    52658829
True      3815053
Name: yld_pt, dtype: int64
rpt_side_cd
False    56473882
Name: rpt_side_cd, dtype: int64
sub_prdct
False    56473882
Name: sub_prdct, dtype: int64


In [55]:
final_df = final_df[final_df['yld_pt'].notna()]
final_df.shape

(52658829, 10)

In [56]:
final_df['cusip_id'].nunique()

5228

In [57]:
final_df.to_csv("final_data_all_ver2.csv.gz",compression='gzip',index=False)


In [59]:
final_df.shape

(52658829, 10)

# Trace Data Cleaned Weekly Frequency

In [21]:
final_df['trd_exctn_dt'] = pd.to_datetime(final_df['trd_exctn_dt'])


In [22]:
final_df = final_df.sort_values(by=['cusip_id','trd_exctn_dt','entrd_vol_qt'], ascending=[True,True,False])

final_df



,cusip_id,bond_sym_id,company_symbol,trd_exctn_dt,trd_exctn_tm,entrd_vol_qt,rptd_pr,yld_pt,rpt_side_cd,sub_prdct
1422120,00037BAA0,ABB3852123,ABB,2012-10-01,15:26:58,625000.0,101.849,1.209968,S,CORP
1422119,00037BAA0,ABB3852123,ABB,2012-10-01,11:41:41,510000.0,101.727,1.237081,S,CORP
1422121,00037BAA0,ABB3852123,ABB,2012-10-01,16:02:48,350000.0,101.916,1.195093,S,CORP
1422118,00037BAA0,ABB3852123,ABB,2012-10-01,11:09:23,150000.0,101.795,1.221964,S,CORP
1422117,00037BAA0,ABB3852123,ABB,2012-10-01,5:38:16,15000.0,101.866,1.206192,S,CORP
...,...,...,...,...,...,...,...,...,...,...
3544653,983919AK7,AMD4987233,AMD,2022-09-29,13:55:48,219000.0,82.348,5.197043,S,CORP
3544654,983919AK7,AMD4987233,AMD,2022-09-29,14:06:05,35000.0,82.337,5.199009,S,CORP
3544657,983919AK7,AMD4987233,AMD,2022-09-30,9:19:58,750000.0,82.937,5.093108,B,CORP
3544659,983919AK7,AMD4987233,AMD,2022-09-30,16:00:59,220000.0,82.522,5.166913,B,CORP


In [23]:
final_df[final_df['cusip_id']=='00037BAA0']['trd_exctn_dt'].nunique()

733

In [24]:

# Here we take the first row on the basis of the volume for each date, 
# if multiple trades for the same bond exist on the same day then we consider 
# the trade with the highest volume
grouped_df = final_df.groupby(['trd_exctn_dt', 'cusip_id']).first().reset_index()

print(grouped_df.shape)


(1738717, 10)


In [28]:
grouped_df['cusip_id'].value_counts()

05968LAH5    2485
023135AJ5    2472
12686CBB4    2459
126650BZ2    2438
459745GN9    2410
             ... 
90131HBP9       3
71647NAF6       3
71647NAB5       3
71647NAC3       2
674599DS1       2
Name: cusip_id, Length: 2237, dtype: int64

In [13]:
grouped_df[grouped_df['cusip_id']=='00037BAA0']['trd_exctn_dt']

0        2012-10-01
96       2012-10-02
198      2012-10-03
514      2012-10-09
728      2012-10-11
            ...    
432656   2017-04-25
433320   2017-04-26
433969   2017-04-27
435238   2017-05-01
435855   2017-05-02
Name: trd_exctn_dt, Length: 733, dtype: datetime64[ns]

In [14]:
grouped_df['cusip_id'].nunique()

2237

In [48]:
grouped_df.columns

Index(['trd_exctn_dt', 'cusip_id', 'bond_sym_id', 'company_symbol',
       'trd_exctn_tm', 'entrd_vol_qt', 'rptd_pr', 'yld_pt', 'rpt_side_cd',
       'sub_prdct'],
      dtype='object')

In [49]:
grouped_df.head()

,trd_exctn_dt,cusip_id,bond_sym_id,company_symbol,trd_exctn_tm,entrd_vol_qt,rptd_pr,yld_pt,rpt_side_cd,sub_prdct
0,2012-10-01,00037BAA0,ABB3852123,ABB,15:26:58,625000.0,101.849,1.209968,S,CORP
1,2012-10-01,001055AH5,AFL3818464,AFL,16:20:20,1801000.0,105.190,1.419056,S,CORP
2,2012-10-01,001546AM2,AKS3832861,AKS,16:12:11,60000.0,85.900,10.784508,S,CORP
3,2012-10-01,00206RBB7,T3818498,T,10:08:19,1900000.0,100.758,0.550999,B,CORP
4,2012-10-01,00206RBC5,T3818485,T,11:00:42,2089000.0,102.700,0.966562,B,CORP


In [50]:


# Group by 'cusip_id' and resample each group to monthly frequency
monthly_grouped_dfs = []
for cusip_id, group in grouped_df.groupby('cusip_id'):
    group.set_index('trd_exctn_dt', inplace=True)
    monthly_grouped_df = group.resample('W').last().reset_index()
    monthly_grouped_dfs.append(monthly_grouped_df)

# Concatenate all the monthly grouped DataFrames back together
result_df = pd.concat(monthly_grouped_dfs, ignore_index=True)
result_df.reset_index(inplace=True)
result_df.shape


(1106046, 11)

In [51]:
result_df['cusip_id'].nunique()

5228

In [52]:
result_df.to_csv("bigger_final_data.csv.gz",compression='gzip')

In [53]:
result_df['cusip_id'].nunique()

5228

In [54]:
result_df.shape

(1106046, 11)

In [55]:
result_df.columns

Index(['index', 'trd_exctn_dt', 'cusip_id', 'bond_sym_id', 'company_symbol',
       'trd_exctn_tm', 'entrd_vol_qt', 'rptd_pr', 'yld_pt', 'rpt_side_cd',
       'sub_prdct'],
      dtype='object')

In [56]:

# Calculate the count of entries for each 'cusip_id' (bond) and store it in a new column 'entry_count'
result_df['no_of_weeks'] = result_df.groupby('cusip_id')['cusip_id'].transform('count')

# Filter out all the bonds with more than 200 entries
df_w_200 = result_df[result_df['no_of_weeks'] > 200]

# Drop the 'entry_count' column if it's not needed anymore
df_w_200.drop(columns='no_of_weeks', inplace=True)

df_w_200.shape


/var/tmp/ipykernel_98788/3561484508.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_200.drop(columns='no_of_weeks', inplace=True)


(722763, 11)

In [57]:
df_w_200['cusip_id'].nunique()

2293

In [58]:
df_w_200.to_pickle('final_data_reduced.pkl')

In [59]:
df_w_200['cusip_id'].unique()

array(['00037BAA0', '00037BAB8', '00101JAF3', ..., '98978VAK9',
       '98978VAL7', '98978VAN3'], dtype=object)

In [60]:


# Group by 'trd_exctn_dt' and count the number of unique 'cusip_id' values in each group
bonds_per_date = df_w_200.groupby('trd_exctn_dt')['cusip_id'].nunique().reset_index()
bonds_per_date.columns = ['trd_exctn_dt', 'num_bonds_per_date']

print(bonds_per_date)


    trd_exctn_dt  num_bonds_per_date
0     2012-10-07                 231
1     2012-10-14                 235
2     2012-10-21                 243
3     2012-10-28                 246
4     2012-11-04                 243
..           ...                 ...
517   2022-09-04                1267
518   2022-09-11                1254
519   2022-09-18                1256
520   2022-09-25                1259
521   2022-10-02                1253

[522 rows x 2 columns]
